In [1]:
#ライブラリインポート

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
from tqdm.notebook import tqdm
import itertools
import requests

In [2]:
#レースIDのリスト作成
#URL構造　"https://db.netkeiba.com/race/"　+　race_id
#race_id構造　年,開催場所,回数,何日目か,レース

race_id_list = []
for year in range(2008,2022):
    #(2008,2021)
    for place in range(1, 11):
        #(1, 11)
        for kai in range(1, 7):
            #(1,7)
            for day in range(1, 13):
                #(1,13)
                for r in range(1, 13):
                    #(1,13)
                    race_id = str(year).zfill(4) + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2)
                    race_id_list.append(race_id)


In [3]:
#レース結果スクレイピング
def get_results(old_df, race_id):
    url = "https://db.netkeiba.com/race/" + race_id

    try:
        ## Create object page
        page = requests.get(url)
        
        ## parser-lxml = Change html to Python friendly format
        ## Obtain page's information
        ## soup = BeautifulSoup(page.text, 'lxml')
        soup = BeautifulSoup(page.content, 'lxml')
        # soup

        ## Obtain information from tag <table>
        table1 = soup.find('table', class_='race_table_01 nk_tb_common')
        # table1

        ## Obtain every title of columns with tag <th>
        headers = ['race_id']
        for i in table1.find_all('th'):
            title = i.text
            headers.append(title)

        ## Create a dataframe
        df = pd.DataFrame(columns = headers)

        for j in table1.find_all('tr')[1:]:
            row_data = j.find_all('td')
            row = [i.text.rstrip() for i in row_data]
            ## Make first column race id
            row = [[race_id], row]
            row = list(itertools.chain.from_iterable(row))
            
            length = len(df)
            df.loc[length] = row

        df['馬名'] = df['馬名'].apply(lambda x: x.split('\n')[1])
        df['騎手'] = df['騎手'].apply(lambda x: x.split('\n')[1])

        # 着順に数字以外の文字列が含まれているものを取り除く
        df = df[~(df["着順"].astype(str).str.contains("\D"))]
        df["着順"] = df["着順"].astype(int)
        
        # 性齢を性と年齢に分ける
        df["性"] = df["性齢"].map(lambda x: str(x)[0])
        df["年齢"] = df["性齢"].map(lambda x: str(x)[1:]).astype(int)

        # 馬体重を体重と体重変化に分ける
        df["体重"] = df["馬体重"].str.split("(", expand=True)[0].astype(int)
        df["体重変化"] = df["馬体重"].str.split("(", expand=True)[1].str[:-1].astype(int)

        # データをint, floatに変換
        df["単勝"] = df["単勝"].astype(float)

        # 不要な列を削除
        df = df.drop(['ﾀｲﾑ指数', '通過', '着差', '性齢', '馬体重', '調教ﾀｲﾑ', '厩舎ｺﾒﾝﾄ', '備考', '調教師',
        '馬主', '賞金(万円)'], axis=1)

        new_df = pd.concat([old_df, df])
        return new_df
    
    except:
        return old_df

In [4]:
##　空のデータフレーム作成
headers = ['race_id', '着順', '枠番', '馬番', '馬名', '斤量', '騎手', 'タイム', '上り', '単勝', '人気',
       '性', '年齢', '体重', '体重変化']x
old_df = pd.DataFrame(columns = headers)

In [5]:
## 指定のレース結果全て１つのデータフレームにする
for race in tqdm(race_id_list):
    new_df = get_results(old_df, race)
    old_df = new_df

  0%|          | 0/120960 [00:00<?, ?it/s]

In [6]:
new_df

,race_id,着順,枠番,馬番,馬名,斤量,騎手,タイム,上り,単勝,人気,性,年齢,体重,体重変化
0,200801010101,1,4,6,ディアジーナ,54,四位洋文,1:31.0,35.6,4.3,2,牝,2,468,0
1,200801010101,2,8,14,アイディンパワー,54,藤田伸二,1:31.4,35.9,5.2,4,牡,2,460,0
2,200801010101,3,1,1,ウインリーズン,54,安藤勝己,1:31.5,36.4,3.1,1,牡,2,442,2
3,200801010101,4,7,11,ヴァンエボン,54,小林徹弥,1:31.6,35.5,23.0,8,牡,2,482,-4
4,200801010101,5,2,2,ラヴェリータ,54,長谷川浩,1:31.6,35.7,21.9,6,牝,2,484,-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,202110040812,7,2,2,ジオルティ,51,角田大和,2:42.0,35.3,49.6,10,牡,3,502,8
7,202110040812,8,7,9,ロックグラス,57,幸英明,2:42.2,35.8,43.7,8,牡,4,464,0
8,202110040812,9,7,8,サマーカナロア,53,富田暁,2:42.2,35.4,46.5,9,牡,3,502,-10
9,202110040812,10,8,10,レッドレイル,57,浜中俊,2:42.4,36.3,14.0,6,セ,5,436,2


In [7]:
#CSVファイルで出力
new_df.to_csv("race_results.csv",encoding="shift_jis")